In [1]:
import os
os.getcwd()

'/opt/workspace/Projet'

In [2]:
import pyspark
from pyspark import SparkContext
!pip install imageio
import imageio
import os
import numpy as np

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
def readImg(path):
    img = imageio.imread(path)
    im = np.array(img,dtype='uint8')
    return im

def writeImg(path,buf):
    imageio.imwrite(path,buf)

def part_median_filterV1(local_data):
    #print(local_data)
    part_id = local_data[0]
    first   = local_data[1]
    end     = local_data[2]
    buf     = local_data[3]
    nx=buf.shape[0]
    ny=buf.shape[1]
    #print(buf[0])
    #print(first,"first\n")
    #print(end,"end\n")
    #print(part_id,"part_id,\n")
    #print(type(buf))
    
    ########################################
    #
    # CREATE NEW BUF WITH MEDIAN FILTER SOLUTION #juste créer le buffer vide
    #
    #new_buf=np.array([end-first,ny],dtype='uint8')
    #new_buf=buf
    #new_buf.shape(nx,ny)
    #print(buf,"buf\n")
    #print(new_buf,"new_buf\n")
    #print("buf",buf[0])
    #print("nbuf",new_buf[0])
    #new_buf[0]=buf[0]
    #new_buf[nx/8,ny/8] = []
    #new_buf = buf.shape
    #new_buf = new_buf.reshape(nx/8,ny/8,3)
    new_buf = buf[first:end]
    print(new_buf)
    print(new_buf.shape)
    sleep(1000)
    #new_buf=buf
    
    ##########################################
    #
    # TODO COMPUTE MEDIAN FILTER
    #
    #from scipy import ndimage
    #part_id = ndimage.median_filter(part_id, size=20)
    #for i in range(1,100):
        #new_buf[i]=buf[i-end]+buf[i+end]+buf[i-end+first]+buf[i-end-first]+buf[i+end+first]+buf[i+end-first]+buf[i-first]+buf[i+first]+buf[i]
        #new_buf[i]=buf[i-1]+buf[i+1]+buf[i-1+1]+buf[i-1-1]+buf[i+1+1]+buf[i+1-1]+buf[i-1]+buf[i+1]+buf[i]
        #new_buf[i]=buf[i-1]
    
    
    ##########################################
    #
    # RETURN LOCAL IMAGE PART
    #
    #print(new_buf.shape)
    return part_id,new_buf

In [4]:
data_dir="./data/"
file = os.path.join(data_dir,'lena_noisy.jpg')
img_buf=readImg(file)
#print('SHAPE',img_buf.shape)
#print('IMG\n',img_buf)
nx=img_buf.shape[0]
ny=img_buf.shape[1]

###########################################################################
#
# SPLIT IMAGES IN NB_PARTITIONS PARTS
nb_partitions = 8
print("NB PARTITIONS : ",nb_partitions)
data=[]
begin=0
block_size=nx/nb_partitions
for ip in range(nb_partitions):
    end=min(begin+block_size,nx)
    data.append([ip,begin,end,img_buf])
    begin=end


###########################################################################
#
# CREATE SPARKCONTEXT
sc =SparkContext()
data_rdd = sc.parallelize(data,nb_partitions)	

NB PARTITIONS :  8


22/02/21 12:21:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/21 12:21:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/02/21 12:21:39 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/02/21 12:21:39 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:
def part_median_filter(local_data):
    #print(local_data)
    part_id = local_data[0]
    first   = local_data[1]
    end     = local_data[2]
    buf     = local_data[3]
    nx=buf.shape[0]
    ny=buf.shape[1]
    
    index = (part_id*ny//nb_partitions)
    if part_id == 0:
        temp = img_buf[index:(index)+ny//nb_partitions+1].copy()
    elif part_id == nb_partitions-1:
        temp = img_buf[index-1:(index)+ny//nb_partitions].copy()
    else:
        temp = img_buf[index-1:(index)+ny//nb_partitions+1].copy()
    #temp = img_buf[index:index+ny//nb_partitions].copy()
    #temp = img_buf[index-1:(index)+ny//nb_partitions+1].copy()
    #print(part_id,index,index+ny//nb_partitions)
    #print(part_id)
    #new_buf = temp.copy()
    new_buf = np.zeros(temp.shape)
    print(temp.shape)
    #print(part_id)
    #print(part_id,new_buf)
    #print(index)
    #print(index+ny//8)
    
    #new_img_buf = img_buf.copy()
    for i in range(0,temp.shape[0]-1):
        for j in range(0,temp.shape[1]-1):
            #print("old",img_buf[i,j])
            #new_img_buf[i, j, :] = np.array([10., 2., -4.])
            #new_img_buf[i,j]=img_buf[i,j]
            #new_img_buf[i,j]=(img_buf[i,j]+img_buf[i+1,j]+img_buf[i,j+1]+img_buf[i+1,j+1]+img_buf[i-1,j]+img_buf[i,j-1]+img_buf[i-1,j-1]+img_buf[i+1,j-1]+img_buf[i-1,j+1])
            #if i == temp.shape[0]-1:
            #    new_buf[i,j]=np.median([temp[i,j,:],temp[i+1,j,:],temp[i,j+1,:],temp[i+1,j+1,:]],axis=0)
            #elif j == temp.shape[1]-1:
            #    new_buf[i,j]=np.median([temp[i,j,:],temp[i-1,j,:],temp[i,j-1,:],temp[i-1,j-1,:]],axis=0)
            #else:
            #    new_buf[i,j]=np.median([temp[i,j,:],temp[i+1,j,:],temp[i,j+1,:],temp[i+1,j+1,:],temp[i-1,j,:],temp[i,j-1,:],temp[i-1,j-1,:],temp[i+1,j-1,:],temp[i-1,j+1,:]],axis=0)
            new_buf[i,j]=np.median([temp[i,j,:],temp[i+1,j,:],temp[i,j+1,:],temp[i+1,j+1,:],temp[i-1,j,:],temp[i,j-1,:],temp[i-1,j-1,:],temp[i+1,j-1,:],temp[i-1,j+1,:]],axis=0)
        new_buf[i,temp.shape[1]-1]=np.median([temp[i,j,:],temp[i,j+1,:],temp[i-1,j,:],temp[i,j-1,:],temp[i-1,j-1,:],temp[i-1,j+1,:]],axis=0)
        
    
    if part_id == 0:
        new_buf = new_buf[:-1].copy()
    elif part_id == nb_partitions-1:
        new_buf = new_buf[1:].copy()
        for j in range(0,temp.shape[1]-1):
            new_buf[ny//nb_partitions-1,j]=np.median([temp[i,j,:],temp[i+1,j,:],temp[i-1,j,:],temp[i,j-1,:],temp[i-1,j-1,:],temp[i+1,j-1,:]],axis=0)
    else:
        new_buf = new_buf[1:-1].copy()
    
    return part_id,new_buf

In [6]:
###########################################################################
#
# PARALLEL MEDIAN FILTER COMPUTATION
result_rdd = data_rdd.map(part_median_filter)
result_data = result_rdd.collect()

#new_img_buf=np.array([nx,ny],dtype='uint8')
#new_img_buf=buf

(18, 128, 3)                                                        (0 + 8) / 8]
(18, 128, 3)
(18, 128, 3)
(17, 128, 3)
(18, 128, 3)
(17, 128, 3)
(18, 128, 3)
(18, 128, 3)


In [7]:
img_buf.shape

(128, 128, 3)

In [8]:
#new_buf.shape

In [9]:
len(result_data)

8

In [10]:
#là

In [11]:
#img_buf[0]

In [12]:
#result_data[0][1]

def another(imgBuf):
    # global new_img_buf
    new_img_buf = np.zeros(imgBuf.shape)
    for i in range(1,imgBuf.shape[0]-1):
        for j in range(1,imgBuf.shape[1]-1):
            # print("old",imgBuf[i,j])
            new_img_buf[i, j, :] = np.array([10., 2., -4.])
            # new_img_buf[i,j, :]=(imgBuf[i,j, :]+imgBuf[i+1,j, :]+imgBuf[i,j+1, :]+imgBuf[i+1,j+1, :]+imgBuf[i-1,j, :]+imgBuf[i,j-1, :]+imgBuf[i-1,j-1,:]+imgBuf[i+1,j-1,:]+imgBuf[i-1,j+1,:])//9
            # print("new",new_img_buf[i,j])
    return new_img_buf

In [13]:
#new_img_buf = another(img_buf)
#print(img_buf.shape)
#print(new_img_buf.shape)
# new_img_buf = img_buf
# another(img_buf)

#new_img_buf = np.zeros(img_buf.shape)
new_img_buf = img_buf.copy()
for i in range(1,img_buf.shape[0]-1):
    for j in range(1,img_buf.shape[1]-1):
        #print("old",img_buf[i,j])
        #new_img_buf[i, j, :] = np.array([10., 2., -4.])
        #new_img_buf[i,j]=img_buf[i,j]
        #new_img_buf[i,j]=(img_buf[i,j]+img_buf[i+1,j]+img_buf[i,j+1]+img_buf[i+1,j+1]+img_buf[i-1,j]+img_buf[i,j-1]+img_buf[i-1,j-1]+img_buf[i+1,j-1]+img_buf[i-1,j+1])
        new_img_buf[i,j]=np.median([img_buf[i,j,:],img_buf[i+1,j,:],img_buf[i,j+1,:],img_buf[i+1,j+1,:],img_buf[i-1,j,:],img_buf[i,j-1,:],img_buf[i-1,j-1,:],img_buf[i+1,j-1,:],img_buf[i-1,j+1,:]],axis=0)
        #print("new",new_img_buf[i,j])

In [14]:
###########################################################################
#
# COMPUTE NEW IMAGE RESULTS FROM RESULT RDD
# TODO
#begin=0
#for ip in range(nb_partitions):
    #end=min(begin+block_size,nx)
    #new_img_buf.append([ip,begin,end,result_data])
    #np.append(new_img_buf, [ip,begin,end,result_data])
    #np.append(new_img_buf, [ip,begin,end,result_data[ip]])
    #new_img_buf.append([ip,begin,end,result_data[ip]])
    #new_img_buf.append(result_data[ip])
    #begin=end

new_img_buf = []
for part in result_data:
    new_img_buf.append(part[1])
    print(part[1].shape)
    #print("\n\n\n")
    #np.append(new_img_buf, part[1])
    #print(new_img_buf)
#print(new_img_buf[0][1])
    
#new_img_buf = np.array(result_data)

new_img_buf = np.vstack(new_img_buf)
    
print('CREATE NEW PICTURE FILE')
filter_file = os.path.join(data_dir,'lena_filter.jpg')
writeImg(filter_file,new_img_buf)

Lossy conversion from float64 to uint8. Range [0.0, 255.0]. Convert image to uint8 prior to saving to suppress this warning.


(16, 128, 3)
(16, 128, 3)
(16, 128, 3)
(16, 128, 3)
(16, 128, 3)
(16, 128, 3)
(16, 128, 3)
(16, 128, 3)
CREATE NEW PICTURE FILE


In [ ]:
new_img_buf.shape

In [ ]:
len(new_img_buf)

In [ ]:
test = np.vstack(new_img_buf)

In [ ]:
test.shape

In [ ]:
part[1]

In [ ]:
new_img_buf[1,1]

In [ ]:
new_img_buf[2,2]

In [ ]:
(new_img_buf[1,1] + new_img_buf[2,2])//2

In [ ]:
test = np.sum(new_img_buf,axis=1)

In [ ]:
np.median([img_buf[i,j,:],img_buf[i+1,j,:],img_buf[i,j+1,:],img_buf[i+1,j+1,:],img_buf[i-1,j,:],img_buf[i,j-1,:],img_buf[i-1,j-1,:],img_buf[i+1,j-1,:],img_buf[i-1,j+1,:]],axis=0)

In [ ]:
img_buf[i,j,:]

In [ ]:
(img_buf[i,j]+img_buf[i+1,j]+img_buf[i,j+1]+img_buf[i+1,j+1]+img_buf[i-1,j]+img_buf[i,j-1]+img_buf[i-1,j-1]+img_buf[i+1,j-1]+img_buf[i-1,j+1])

In [ ]:
print(img_buf[i,j])
print(img_buf[i+1,j])
print(img_buf[i,j+1])
print(img_buf[i-1,j])
print(img_buf[i,j-1])
print(img_buf[i+1,j+1])
print(img_buf[i+1,j-1])
print(img_buf[i-1,j+1])
print(img_buf[i-1,j-1])

In [ ]:
(118+125+122+104+100+214+109+107+131)//9

In [ ]:
(82+94+90+55+56+190+69+61+78)//9

In [ ]:
#img_buf[1,1]

In [ ]:
#img_buf.shape[0]

In [ ]:
#img_buf[1]

In [ ]:
#new_img_buf[1]

In [ ]:
#new_img_buf[2]

In [ ]:
#img_buf[2]

In [ ]:
img_buf[1,1]

In [ ]:
type(new_img_buf)

In [ ]:
type(img_buf)

In [ ]:
new_img_buf.shape

In [ ]:
img_buf.shape

In [ ]:
test = new_img_buf.flatten()

In [ ]:
test.shape

In [ ]:
result_data[0][1]

In [ ]:
a = np.zeros(img_buf.shape)
a[0, 0, :] = np.array([10., 2., -4.])
print(a[0, 0])